# Singular Value Decomposition

What it does?
- reduce dimensionality of data
- remove noises
- may improve algorithm results
- compresses data

Use Cases:
- recommendation engine (Netflix prize)
- image compression

In [321]:
import numpy as np

In [322]:
u, sigma, vt = np.linalg.svd([[1,1], [7,7]])
u

array([[-0.14142136, -0.98994949],
       [-0.98994949,  0.14142136]])

In [323]:
sigma

array([  1.00000000e+01,   2.82797782e-16])

In [324]:
vt

array([[-0.70710678, -0.70710678],
       [ 0.70710678, -0.70710678]])

In [325]:
def load_data():
    return [[1, 1, 1, 0, 0],
            [2, 2, 2, 0, 0],
            [1, 1, 1, 0, 0],
            [5, 5, 5, 0, 0],
            [1, 1, 0, 2, 3],
            [0, 0, 0, 3, 3],
            [0, 0, 0, 1, 1]]

In [326]:
data = load_data()
u, sigma, vt = np.linalg.svd(data)
sigma

array([  9.72757334e+00,   5.71597993e+00,   8.37789008e-01,
         1.19631129e-15,   6.55271165e-16])

In [327]:
# How many singular values to keep? Keep 90% of it.
# To calculate the total energy, add up all the squared singular value until you reach 90% of the value.

total_energy = np.sum(sigma)
current_energy = 0
singular_values = 0
for i, v in enumerate(sigma):
    current_energy += v
    if current_energy/total_energy >= 0.9:
        singular_values = i + 1
        break

singular_values, current_energy

(2, 15.443553264742762)

In [328]:
# sig3 = np.mat([[sigma[0], 0, 0], [0, sigma[1], 0], [0, 0, sigma[2]]])
# sig3

In [329]:
sig3 = np.mat(np.diag(sigma[:3]))
sig3

matrix([[ 9.72757334,  0.        ,  0.        ],
        [ 0.        ,  5.71597993,  0.        ],
        [ 0.        ,  0.        ,  0.83778901]])

In [330]:
u[:,:3] * sig3 * vt[:3,:]

matrix([[  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          -1.09705447e-16,  -4.76181594e-16],
        [  2.00000000e+00,   2.00000000e+00,   2.00000000e+00,
           1.29675384e-16,   1.63064007e-16],
        [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
           1.15307731e-15,   2.49800181e-15],
        [  5.00000000e+00,   5.00000000e+00,   5.00000000e+00,
          -4.67654843e-17,   9.71445147e-17],
        [  1.00000000e+00,   1.00000000e+00,   5.71838092e-16,
           2.00000000e+00,   3.00000000e+00],
        [ -9.33966978e-17,  -3.43623221e-16,   7.05773396e-16,
           3.00000000e+00,   3.00000000e+00],
        [ -2.77555756e-17,  -1.31838984e-16,   2.35922393e-16,
           1.00000000e+00,   1.00000000e+00]])

In [331]:
def euclidean(a, b):
    return 1.0 / (1.0 + np.linalg.norm(a - b))

def pearson(a, b):
    if len(a) < 3: return 1
    return 0.5 + 0.5 * np.corrcoef(a, b, rowvar = 0)[0][1]

def cosine(a, b):
    num = float(a.T * b)
    den = np.linalg.norm(a) * np.linalg.norm(b)
    return 0.5 + 0.5 * num / den

In [332]:
data = np.mat(load_data())
euclidean(data[:, 0], data[:, 4])

0.12973190755680383

In [333]:
euclidean(data[:, 0], data[:, 0])

1.0

In [334]:
cosine(data[:, 0], data[:, 4])

0.56083303292403597

In [335]:
cosine(data[:, 0], data[:, 0])

0.99999999999999989

In [336]:
pearson(data[:, 0], data[:, 4])

0.25994232323867872

In [337]:
pearson(data[:, 0], data[:, 0])

1.0

In [338]:
def standard_estimator(data, user, similarity_class, item):
    n = np.shape(data)[1]
#     print('shape:', np.shape(data))
    similarity_score = 0
    rating_similarity_score = 0
    for j in range(n):
        user_rating = data[user, j]
#         print('user_rating: {}'.format(user_rating))
        if user_rating == 0:
            continue
        
#         print('item and column: {} {}'.format(data[:, item], data[:,j]))
        overlap = np.nonzero(np.logical_and(data[:, item].A > 0,
                                            data[:, j].A > 0))[0]
#         print('overlap: {}'.format(overlap))
        
        if len(overlap) == 0:
            score = 0
        else:
            score = similarity_class(data[overlap, item], data[overlap, j])
        
        print('the {} and {} similarity is {}'.format(item, j, score))
        print()
        
        similarity_score += score
        rating_similarity_score += score * user_rating
    if similarity_score == 0:
        return 0
    else:
        return rating_similarity_score / similarity_score

In [339]:
def recommend(data, user, N = 3, similarity_class = cosine, estimator = standard_estimator):
    unrated_items = np.nonzero(data[user, :].A == 0)[1]
    if len(unrated_items) == 0:
        return 'you rated everything'
    item_scores = []
    for item in unrated_items:
        estimated_score = estimator(data, user, similarity_class, item)
        item_scores.append((item, estimated_score))
    return sorted(item_scores, key = lambda x: x[1], reverse = True)[:N]

In [340]:
data = np.matrix([[4, 4, 0, 2, 2],
                   [4, 0, 0, 3, 3],
                   [4, 0, 0, 1, 1],
                   [1, 1, 1, 2, 0],
                   [2, 2, 2, 0, 0],
                   [1, 1, 1, 0, 0],
                   [5, 5, 5, 0, 0]])

In [341]:
recommend(data, 2)

the 1 and 0 similarity is 1.0

the 1 and 3 similarity is 0.928746462856272

the 1 and 4 similarity is 1.0

the 2 and 0 similarity is 1.0

the 2 and 3 similarity is 1.0

the 2 and 4 similarity is 0



[(2, 2.5), (1, 2.0243290220056256)]

In [344]:
def svd_estimator(data, user, similarity_class, item):
    n = np.shape(data)[1]
    similarity_total = 0
    rating_similarity_total = 0
    u, sigma, vt = np.linalg.svd(data)
    sig4 = np.mat(np.eye(4) * sigma[:4])
    xformed_items = data.T * u[:, :4] * sig4.I
    for j in range(n):
        user_rating = data[user, j]
        if user_rating == 0 or j == item:
            continue
        score = similarity_class(xformed_items[item, :].T, 
                                 xformed_items[j, :].T)
        print('the {} and {} similarity is {:.3f}'.format(item, j, score))
        similarity_total += score
        rating_similarity_total += score * user_rating
    if similarity_total == 0:
        return 0
    else:
        return rating_similarity_total / similarity_total

In [371]:
data = np.mat([[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
               [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
               [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
               [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
               [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
               [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
               [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
               [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
               [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
               [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
               [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]])

In [376]:
u, sigma, vt = np.linalg.svd(np.mat(data))
sigma_square = sigma ** 2
target = np.sum(sigma_square) * 0.9
print('target: {}'.format(target))
np.sum(sigma_square[:3])

target: 487.7999999999993


500.50028912757909

In [377]:
recommend(data = data, user = 1, estimator = svd_estimator, similarity_class = cosine)

the 0 and 3 similarity is 0.491
the 0 and 5 similarity is 0.484
the 0 and 10 similarity is 0.513
the 1 and 3 similarity is 0.491
the 1 and 5 similarity is 0.482
the 1 and 10 similarity is 0.510
the 2 and 3 similarity is 0.492
the 2 and 5 similarity is 0.482
the 2 and 10 similarity is 0.511
the 4 and 3 similarity is 0.450
the 4 and 5 similarity is 0.507
the 4 and 10 similarity is 0.513
the 6 and 3 similarity is 0.744
the 6 and 5 similarity is 0.468
the 6 and 10 similarity is 0.439
the 7 and 3 similarity is 0.482
the 7 and 5 similarity is 0.495
the 7 and 10 similarity is 0.525
the 8 and 3 similarity is 0.491
the 8 and 5 similarity is 0.491
the 8 and 10 similarity is 0.520
the 9 and 3 similarity is 0.522
the 9 and 5 similarity is 0.496
the 9 and 10 similarity is 0.494


[(4, 3.3447149384692278), (7, 3.3294020724526971), (9, 3.3281008763900686)]